In [ ]:
!pip install tensorflow-addons

# **Aumento de datos**

In [ ]:
import shutil
import random
import cv2
import glob
import numpy as np
import os

Extraer nombre de las carpetas

In [ ]:
import json
with open('imagenet_vid_train_15frames.json', 'r') as file:
    data = json.load(file)


# Extraer los nombres de las carpetas y añadir "/content/" al principio de cada uno
folder_names = ["/content/" + video["name"] for video in data["videos"]]

significant_frames1 = []

for video in data["videos"]:
    frames = video["vid_train_frames"]
    frames = [frame for frame in frames if frame != -1]  # Quitar -1
    significant_frames1.append(frames)


#print(folder_names)

# Imprimir las listas en sublistas de una más grande
#print(significant_frames1)

['/content/benign/x28f299ceb056964c', '/content/benign/7de856d2db6d4700', '/content/benign/x717d46eb599764e', '/content/malignant/355f8098c8aa52ed', '/content/malignant/440c1dc384492d08', '/content/benign/x4771d87a9fdeb83b', '/content/benign/x32a4159934e524d5', '/content/benign/666818cd5f46c701', '/content/malignant/x27a0646da982181b', '/content/malignant/58534f86429d0af9', '/content/malignant/x460c43be47ad80b9', '/content/benign/531bdc37158304f9', '/content/malignant/x465f73e745809528', '/content/benign/6085475b648ee8f7', '/content/benign/32a6532065f51cec', '/content/malignant/4dc30d4e51ee2e48', '/content/benign/786ebd9dada09453', '/content/malignant/5b01151520267051', '/content/benign/x3748594d89375c56', '/content/benign/xff3777c79744514', '/content/malignant/x7cd4703a73a52918', '/content/malignant/69f4b1d2f756e60e', '/content/malignant/x53ce7f0e44720712', '/content/malignant/x2c7abb4741f95ad3', '/content/malignant/x5625e0d7d5570bc2', '/content/benign/x2be0b0e1c5d5aa61', '/content/ma

149

Definir las transformaciones

In [ ]:
def vertical_flip(image):
    return cv2.flip(image, 0)

def rotate(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))  # Asegúrate de usar (w, h)
    return rotated


def translate(image, x, y):
    M = np.float32([[1, 0, x], [0, 1, y]])
    translated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))  # Mantener las dimensiones originales
    return translated


def perspective_transform(image):
    (h, w) = image.shape[:2]
    src_points = np.float32([[0, 0], [w-2, 0], [0, h-3], [w-7, h-5]])
    dst_points = np.float32([[0, 0], [w-8, 0], [0, h-2], [w-3, h-5]])  # Mantener los puntos originales
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    warped = cv2.warpPerspective(image, M, (w, h))  # Mantener las dimensiones originales
    return warped


Aumento de datos

In [ ]:
# Definir función para aplicar transformaciones a las imágenes
def apply_transformations(image_folder):
    # Lista de archivos de imagen, ordenada por nombre
    image_files = sorted(glob.glob(os.path.join(image_folder, '*.png')))  # Cambia la extensión si es necesario

    # Cargar las imágenes
    images = [cv2.imread(image_file) for image_file in image_files]

    # Definir ángulos y desplazamientos
    angle = random.randint(90, 360)  # Ejemplo de rotación
    x_translation = random.randint(10, 50)  # Ejemplo de traslación en x
    y_translation = random.randint(10, 50)  # Ejemplo de traslación en y

    # Aplicar las mismas transformaciones a todas las imágenes
    transformed_images = []
    for image in images:
        img = vertical_flip(image)
        img = rotate(img, angle)
        img = translate(img, x_translation, y_translation)
        img = perspective_transform(img)
        transformed_images.append(img)

    # Crear una carpeta para guardar las imágenes transformadas
    output_folder = os.path.join('/content/transformed_images', os.path.basename(image_folder))
    os.makedirs(output_folder, exist_ok=True)

    # Guardar las imágenes transformadas
    for i, img in enumerate(transformed_images):
        cv2.imwrite(os.path.join(output_folder, f'image_{i}.png'), img)

    # Comprimir la carpeta
    shutil.make_archive(output_folder, 'zip', output_folder)

    # Descargar la carpeta comprimida
    files.download(f'{output_folder}.zip')

# Iterar sobre cada carpeta en folder_names y aplicar las transformaciones
for folder_name in folder_names:
    image_folder = os.path.join(folder_names)
    apply_transformations(image_folder)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Para unir todas las carpetas de imágenes

In [ ]:
import os

# Directorio de interés
directory = "/content/transformed_images"

# Iterar sobre los elementos en el directorio
for item in os.listdir(directory):
    # Comprobar si es una carpeta y si comienza con '/content/transformed_images'
    if os.path.isdir(os.path.join(directory, item)):
        # Agregar el nombre de la carpeta a la lista
        folder_names.append('/content/transformed_images/'+ item)

# print(folder_names)

significant_frames1 = significant_frames1*2
#print(significant_frames1)

# Transformar la lista
significant_frames = []

for sublista in significant_frames1:
    nueva_sublista = [f"{num:06}.png" for num in sublista]
    significant_frames.append(nueva_sublista)


# **Algortimo CNN**

Procesamiento de datos

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array

def load_images_from_folders_with_significant_frames(folder_names, significant_frames):
    images = []
    labels = []

    # Recorrer las carpetas y sus correspondientes frames significativos
    for folder, sig_frames in zip(folder_names, significant_frames):
        if os.path.isdir(folder):
            for filename in os.listdir(folder):
                img_path = os.path.join(folder, filename)
                if os.path.isfile(img_path):
                    img = cv2.imread(img_path)
                    if img is not None:
                        # Redimensionar la imagen si es necesario
                        img = cv2.resize(img, (224, 224))  # Ajusta este tamaño según sea necesario
                        img = img_to_array(img)
                        images.append(img)
                        # Comprobar si el nombre del frame está en la lista de frames significativos
                        label = 1 if filename in sig_frames else 0
                        labels.append(label)
    return np.array(images), np.array(labels)


# Cargar imágenes y etiquetas
X, y = load_images_from_folders_with_significant_frames(folder_names, significant_frames)


In [ ]:
X = X.astype('float32') / 255.0

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar dimensiones
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Construcción del modelo

In [ ]:
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Suponiendo que X_train, y_train, X_test, y_test ya están definidos y preprocesados

# Obtener las dimensiones de la primera imagen
altura, anchura, canales = X_train[0].shape

# Paso 2: Construcción del modelo
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(altura, anchura, canales)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Salida binaria: frame significativo o no
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Paso 3: Configuración de callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model_checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Entrenar el modelo y guardar el historial de entrenamiento
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, model_checkpoint])

# El modelo ya tiene los mejores pesos debido a restore_best_weights=True en EarlyStopping

# Paso 4: Evaluación del modelo
y_pred = model.predict(X_test)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

print(classification_report(y_test, y_pred_binary))

In [ ]:
import matplotlib.pyplot as plt

# Obtener las métricas del historial de entrenamiento
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Crear una figura con dos subgráficos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Subgráfico para la pérdida
ax1.plot(train_loss, label='Training Loss', color='blue')
ax1.plot(val_loss, label='Validation Loss', color='red')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True)

# Subgráfico para la precisión
ax2.plot(train_accuracy, label='Training Accuracy', color='blue')
ax2.plot(val_accuracy, label='Validation Accuracy', color='red')
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True)

# Mostrar la figura
plt.tight_layout()
plt.show()
